In [42]:
import pandas as pd
import numpy as np

In [ ]:
df_wiki_train = pd.read_json('data/QA-dataset/data/2wikimultihopQA/train.json')
df_wiki_dev = pd.read_json('data/QA-dataset/data/2wikimultihopQA/dev.json')
df_wiki_test = pd.read_json('data/QA-dataset/data/2wikimultihopQA/test.json')

df_wiki = pd.concat([df_wiki_train, df_wiki_dev, df_wiki_test])
df_wiki.drop(columns=['type', 'supporting_facts', 'evidences'], inplace=True)

df_wiki = df_wiki.explode('context', ignore_index=True)
df_wiki_corpus = df_wiki['context'].apply(lambda row: row[0] + ' - ' + ' '.join(row[1]))
df_wiki_corpus.drop_duplicates(ignore_index=True, inplace=True)

df_wiki_corpus

In [ ]:
df_musique_train = pd.read_json('data/QA-dataset/data/musique/musique_full_v1.0_train.jsonl', lines=True)
df_musique_dev = pd.read_json('data/QA-dataset/data/musique/musique_full_v1.0_dev.jsonl', lines=True)
df_musique_test = pd.read_json('data/QA-dataset/data/musique/musique_full_v1.0_test.jsonl', lines=True)

df_musique = pd.concat([df_musique_train, df_musique_dev, df_musique_test])
df_musique.drop(columns=['question_decomposition', 'answer_aliases', 'answerable'], inplace=True)

df_musique = df_musique.explode('paragraphs', ignore_index=True)
df_musique_corpus = df_musique['paragraphs'].apply(lambda row: row['title'] + ' - ' +  row['paragraph_text'])
df_musique_corpus.drop_duplicates(ignore_index=True, inplace=True)

df_musique_corpus

In [ ]:
df_corpus = pd.concat([df_wiki_corpus, df_musique_corpus])
df_corpus = df_corpus.sample(frac=1, random_state=42)
df_corpus = df_corpus.reset_index(drop=True)
np.save('data/block_records.npy', df_corpus.to_numpy())
df_corpus